# Evaluation
Whenever calculating tensor products, it is useful to relate them to real word coordinate systems again. Cadabra is able to compute the tensor components for us, provided we specify how the indices map to coordinates.

### Example
Let us specify a coordinate sytem $\{ \theta, \varphi \}$, and map them to our indices:

In [1]:
{\theta, \varphi}::Coordinate.
{a,b,c,d,e,f,g,h#}::Indices(values={\theta, \varphi}, position=independent).

Say we have some vector defined in this system, and wish to calculate the tensor 
$$
d V_{ab} = \partial_b V_a - \partial_a V_b.
$$

In Cadabra, we could specify this as:

In [2]:
\partial{#}::PartialDerivative.

V := { V_{\theta} = \varphi, V_{\varphi} = \sin(\theta) };
dV := \partial_{b}{V_{a}} - \partial_{a}{V_{b}};

${}\left[V_{\theta} = \varphi,~ V_{\varphi} = \sin{\theta}\right]$

${}\partial_{b}{V_{a}}-\partial_{a}{V_{b}}$

Then we evaluate the components with:

In [3]:
evaluate(dV, V);

${}\square{}_{a}{}_{b}\left\{\begin{aligned}\square{}_{\varphi}{}_{\theta}& = \cos{\theta}-1\\[-.5ex]
\square{}_{\theta}{}_{\varphi}& = 1-\cos{\theta}\\[-.5ex]
\end{aligned}\right.
$

There are a few exceptions to what Cadabra allows as valid syntax when it comes to the evaluation operations, and similarly what exactly the evaluate function does can be a little ambiguous. We consult the manual page [here](https://cadabra.science/manual/evaluate.html), where it states

> Evaluate components of a tensor expression.

This means expressions without tensors (i.e. scalars) will not be evaluated on a compotent basis, since there are no scalars to evaluate. We instead would use a call to `sympy`, demonstrated later in the notebook, to perform such an operation.

## Riemann curvature of a 2-sphere
A more involved example is computing the Riemann tensor for a 2-sphere. We begin with a metric in polar coordinates for the squared distance
$$
ds^2 = r^2 \left( d\theta^2 + \sin^2 \left(\theta\right) d\varphi^2 \right).
$$

In [4]:
gab := { 
    g_{\theta \theta} = r**2,
    g_{\varphi \varphi} = r**2 \sin( \theta ) ** 2 
};

${}\left[g_{\theta \theta} = {r}^{2},~ g_{\varphi \varphi} = {r}^{2} {\left(\sin{\theta}\right)}^{2}\right]$

We only need to define the non-zero components in Cadabra. We can now either write in the inverse by hand, on use the `complete` method in Cadabra:

In [5]:
g^{a b}::InverseMetric.
complete(gab, $g^{a b}$);

${}\left[g_{\theta \theta} = {r}^{2},~ g_{\varphi \varphi} = {r}^{2} {\left(\sin{\theta}\right)}^{2},~ g^{\theta \theta} = {r}^{-2},~ g^{\varphi \varphi} = {\left({r}^{2} {\left(\sin{\theta}\right)}^{2}\right)}^{-1}\right]$

Then we define our other rules:

In [6]:
Gamma := \Gamma^{a}_{b c} -> 
      1/2 g^{a d} (\partial_{b}{g_{d c}}
    + \partial_{c}{g_{b d}}
    - \partial_{d}{g_{b c}});
    
Rabcd := R^{a}_{b c d} ->
      \partial_{c}{\Gamma^{a}_{b d}}
    - \partial_{d}{\Gamma^{a}_{b c}}
    + \Gamma^{e}_{b d} \Gamma^{a}_{c e}
    - \Gamma^{e}_{b c} \Gamma^{a}_{d e};

${}\Gamma^{a}\,_{b c} \rightarrow \frac{1}{2}g^{a d} \left(\partial_{b}{g_{d c}}+\partial_{c}{g_{b d}}-\partial_{d}{g_{b c}}\right)$

${}R^{a}\,_{b c d} \rightarrow \partial_{c}{\Gamma^{a}\,_{b d}}-\partial_{d}{\Gamma^{a}\,_{b c}}+\Gamma^{e}\,_{b d} \Gamma^{a}\,_{c e}-\Gamma^{e}\,_{b c} \Gamma^{a}\,_{d e}$

And substitute the requisite tensors

In [7]:
expr := R^{a}_{b c d};
substitute(expr, Rabcd);
substitute(expr, Gamma);

evaluate(expr, gab);

${}R^{a}\,_{b c d}$

${}\partial_{c}{\Gamma^{a}\,_{b d}}-\partial_{d}{\Gamma^{a}\,_{b c}}+\Gamma^{e}\,_{b d} \Gamma^{a}\,_{c e}-\Gamma^{e}\,_{b c} \Gamma^{a}\,_{d e}$

${}\frac{1}{2}\partial_{c}\left(g^{a e} \left(\partial_{b}{g_{e d}}+\partial_{d}{g_{b e}}-\partial_{e}{g_{b d}}\right)\right) - \frac{1}{2}\partial_{d}\left(g^{a e} \left(\partial_{b}{g_{e c}}+\partial_{c}{g_{b e}}-\partial_{e}{g_{b c}}\right)\right)+\frac{1}{4}g^{e f} \left(\partial_{b}{g_{f d}}+\partial_{d}{g_{b f}}-\partial_{f}{g_{b d}}\right) g^{a g} \left(\partial_{c}{g_{g e}}+\partial_{e}{g_{c g}}-\partial_{g}{g_{c e}}\right) - \frac{1}{4}g^{e f} \left(\partial_{b}{g_{f c}}+\partial_{c}{g_{b f}}-\partial_{f}{g_{b c}}\right) g^{a g} \left(\partial_{d}{g_{g e}}+\partial_{e}{g_{d g}}-\partial_{g}{g_{d e}}\right)$

${}\square{}_{d}{}_{b}{}^{a}{}_{c}\left\{\begin{aligned}\square{}_{\varphi}{}_{\varphi}{}^{\theta}{}_{\theta}& = {\left(\sin{\theta}\right)}^{2}\\[-.5ex]
\square{}_{\varphi}{}_{\theta}{}^{\varphi}{}_{\theta}& = -1\\[-.5ex]
\square{}_{\theta}{}_{\varphi}{}^{\theta}{}_{\varphi}& = -{\left(\sin{\theta}\right)}^{2}\\[-.5ex]
\square{}_{\theta}{}_{\theta}{}^{\varphi}{}_{\varphi}& = 1\\[-.5ex]
\end{aligned}\right.
$

Note that despite the indices no longer being in the right order, there is a strict one-to-one correspondance between the input $R^a_{\hphantom{a}bcd}$ and the output indices, such that the generic mapping
$$
a \rightarrow a \\
b \rightarrow b \\
\cdots
$$
exists.

We can easily generalise this approach to calculate, e.g., the Ricci scalar:

In [8]:
expr := g^{c b} R^{a}_{b a c};
substitute(expr, Rabcd);
substitute(expr, Gamma);
evaluate(expr, gab);

${}g^{c b} R^{a}\,_{b a c}$

${}g^{c b} \left(\partial_{a}{\Gamma^{a}\,_{b c}}-\partial_{c}{\Gamma^{a}\,_{b a}}+\Gamma^{e}\,_{b c} \Gamma^{a}\,_{a e}-\Gamma^{e}\,_{b a} \Gamma^{a}\,_{c e}\right)$

${}g^{c b} \left(\frac{1}{2}\partial_{a}\left(g^{a d} \left(\partial_{b}{g_{d c}}+\partial_{c}{g_{b d}}-\partial_{d}{g_{b c}}\right)\right) - \frac{1}{2}\partial_{c}\left(g^{a d} \left(\partial_{b}{g_{d a}}+\partial_{a}{g_{b d}}-\partial_{d}{g_{b a}}\right)\right)+\frac{1}{4}g^{e d} \left(\partial_{b}{g_{d c}}+\partial_{c}{g_{b d}}-\partial_{d}{g_{b c}}\right) g^{a f} \left(\partial_{a}{g_{f e}}+\partial_{e}{g_{a f}}-\partial_{f}{g_{a e}}\right) - \frac{1}{4}g^{e d} \left(\partial_{b}{g_{d a}}+\partial_{a}{g_{b d}}-\partial_{d}{g_{b a}}\right) g^{a f} \left(\partial_{c}{g_{f e}}+\partial_{e}{g_{c f}}-\partial_{f}{g_{c e}}\right)\right)$

${}2{r}^{-2}$

### Selecting specific components
We can use the `get_compontent` to evaluate a specific component of a tensor. For example, the $R^\theta_{\hphantom{\theta}\varphi\theta\varphi}$ compoentn of the Riemann connection on our 2-sphere is:

In [9]:
from cdb.core.component import get_component

 # same as above
expr := R^{a}_{b c d}.
substitute(expr, Rabcd)
substitute(expr, Gamma)
evaluate(expr, gab)

RiemCompt = get_component(expr, $\theta, \varphi, \theta, \varphi$);

We could also approach this by projecting the tensor onto a suitable basis and dual basis, using the composition
$$
R^{a}_{\hphantom{a}bcd} \rightarrow R^{a}_{\hphantom{a}bcd}e^\theta_a e^b_\varphi e^c_\theta e^d_\varphi,
$$
where $e_\theta = \partial_\theta$ is the standard basis for coordinates $\{ \theta, \varphi \}$.

We could define a basis as:

In [10]:
basis := {\theta^{\theta} = 1, \varphi^{\varphi} = 1};
dual  := {\theta_{\theta} = 1, \varphi_{\varphi} = 1};

${}-{\left(\sin{\theta}\right)}^{2}$

${}\left[\theta^{\theta} = 1,~ \varphi^{\varphi} = 1\right]$

${}\left[\theta_{\theta} = 1,~ \varphi_{\varphi} = 1\right]$

And then contract the Riemann tensor with the basis:

In [11]:
compt := R^{a}_{b c d} \theta_{a} \varphi^{b} \theta^{c} \varphi^{d};
substitute(compt, Rabcd);
substitute(compt, Gamma);
evaluate(compt, gab);

${}R^{a}\,_{b c d} \theta_{a} \varphi^{b} \theta^{c} \varphi^{d}$

${}\left(\partial_{c}{\Gamma^{a}\,_{b d}}-\partial_{d}{\Gamma^{a}\,_{b c}}+\Gamma^{e}\,_{b d} \Gamma^{a}\,_{c e}-\Gamma^{e}\,_{b c} \Gamma^{a}\,_{d e}\right) \theta_{a} \varphi^{b} \theta^{c} \varphi^{d}$

${}\left(\frac{1}{2}\partial_{c}\left(g^{a e} \left(\partial_{b}{g_{e d}}+\partial_{d}{g_{b e}}-\partial_{e}{g_{b d}}\right)\right) - \frac{1}{2}\partial_{d}\left(g^{a e} \left(\partial_{b}{g_{e c}}+\partial_{c}{g_{b e}}-\partial_{e}{g_{b c}}\right)\right)+\frac{1}{4}g^{e f} \left(\partial_{b}{g_{f d}}+\partial_{d}{g_{b f}}-\partial_{f}{g_{b d}}\right) g^{a g} \left(\partial_{c}{g_{g e}}+\partial_{e}{g_{c g}}-\partial_{g}{g_{c e}}\right) - \frac{1}{4}g^{e f} \left(\partial_{b}{g_{f c}}+\partial_{c}{g_{b f}}-\partial_{f}{g_{b c}}\right) g^{a g} \left(\partial_{d}{g_{g e}}+\partial_{e}{g_{d g}}-\partial_{g}{g_{d e}}\right)\right) \theta_{a} \varphi^{b} \theta^{c} \varphi^{d}$

${}\theta_{\theta} \theta^{\theta} {\varphi^{\varphi}}^{2} {\left(\sin{\theta}\right)}^{2}-\theta_{\theta} \theta^{\varphi} \varphi^{\theta} \varphi^{\varphi} {\left(\sin{\theta}\right)}^{2}-\theta_{\varphi} \theta^{\theta} \varphi^{\theta} \varphi^{\varphi}+\theta_{\varphi} \theta^{\varphi} {\varphi^{\theta}}^{2}$

## Evaluating with `sympy`
We can convert a cadabra expression to `sympy` using `._sympy_()` -- essentially the counterpart to `Ex()`. For example:

In [12]:
r, theta, varphi = symbols("r theta varphi")
compt := g_{a b};
evaluate(compt, gab);

indcs = compt[2][0][0];
compt = compt[2][0][1];

func = compt._sympy_()

print("Evaluating at r=3 -> " + str(func.subs(r, 3)))

${}g_{a b}$

${}\square{}_{a}{}_{b}\left\{\begin{aligned}\square{}_{\theta}{}_{\theta}& = {r}^{2}\\[-.5ex]
\square{}_{\varphi}{}_{\varphi}& = {r}^{2} {\left(\sin{\theta}\right)}^{2}\\[-.5ex]
\end{aligned}\right.
$

${}\left[\theta,~ \theta\right]$

${}{r}^{2}$

Evaluating at r=3 -> 9


We can then import a helper function from `sympy`:

In [13]:
from sympy import lambdify

And with them, transform our component into a numpy-evaluable function:

In [14]:
f = lambdify(r, func, "numpy")

import numpy as np
print(f(np.arange(10)))

[ 0  1  4  9 16 25 36 49 64 81]
